In [1]:
import os 
import cv2
import numpy as np
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers, models 
import pandas as pd 
path = r'D:\dowlands\automatic-classification-of-fetal-ultrasound-brain\data\data' 
dirs = os.listdir(path)
from PIL import Image

In [2]:
img_data_array = np.zeros((3093,300,300,3))
for img in dirs: 
    image= np.array(Image.open(os.path.join(path, img)))
    image= np.resize(image,(300,300,3))
    image = image.astype('float32')
    image /= 255  
    img_data_array[dirs.index(img)]=image


In [3]:
X_train = img_data_array[:2500]
X_test = img_data_array[2500:]

In [4]:
df= pd.read_csv('train.csv')
dftest=pd.read_csv('sample_submission.csv')

In [5]:
Y = np.array(df['class'].unique())

In [6]:
df['classes'] = df['class'].replace({ 'Trans-thalamic': 0,'Trans-cerebellum': 1,'Trans-ventricular': 2,'Other':3})

In [7]:
Y = np.array(df['classes'])

In [8]:
df['classes'].unique()

array([0, 1, 2, 3], dtype=int64)

In [9]:
Y = tf.keras.utils.to_categorical(
    Y, num_classes=4, dtype='uint8')

In [10]:
Y.shape

(2500, 4)

In [11]:
train_DataGen = tf.keras.preprocessing.image.ImageDataGenerator(zoom_range=0.2, 
                                                                width_shift_range=0.1, 
                                                                height_shift_range = 0.1, 
                                                                horizontal_flip=True)
 
train_set_conv = train_DataGen.flow(X_train, Y, batch_size=16) 

In [12]:
Y_test = df['class']

In [32]:
Y_test.shape

(592, 4)

In [13]:
dftest['class_coded']=dftest['class'].replace({ 'Trans-thalamic': 0,'Trans-cerebellum': 1,'Trans-ventricular': 2,'Other':3})
Y_test = tf.keras.utils.to_categorical(np.array(dftest['class_coded']), num_classes = 4,dtype='uint8')


In [14]:
def happyModel():
    """
    Implements the forward propagation for the binary classification model:
    ZEROPAD2D -> CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> FLATTEN -> DENSE
    """

    model = tf.keras.Sequential([
            ## ZeroPadding2D with padding 3, input shape of 300,300,3
            
            tf.keras.layers.ZeroPadding2D(padding = 3, input_shape = (300,300,3)),
            ## Conv2D with 32 7x7 filters and stride of 1
            tf.keras.layers.Conv2D(32, (7, 7), strides = 1),

            tf.keras.layers.BatchNormalization(),
            ## ReLU

            tf.keras.layers.ReLU(),
            ## Max Pooling 2D with default parameters
            tf.keras.layers.MaxPool2D(),
            ## Flatten layer
            tf.keras.layers.Flatten(),
            ## Dense layer with 4 unit for output & 'sigmoid' activation
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(4, activation = 'sigmoid')
            # YOUR CODE ENDS HERE
        
        ])
    return model
    

In [15]:
happy_model = happyModel()
happy_model.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [16]:
happy_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d (ZeroPadding  (None, 306, 306, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 300, 300, 32)      4736      
                                                                 
 batch_normalization (BatchN  (None, 300, 300, 32)     128       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 300, 300, 32)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 150, 150, 32)     0         
 )                                                               
                                                        

In [17]:
happy_model.fit(train_set_conv, epochs=80, batch_size=16)

Epoch 1/80
157/157 [==============================] - 136s 841ms/step - loss: 11.6045 - accuracy: 0.4108
Epoch 2/80
157/157 [==============================] - 132s 840ms/step - loss: 9.5370 - accuracy: 0.4116
Epoch 3/80
157/157 [==============================] - 131s 831ms/step - loss: 9.2589 - accuracy: 0.3880
Epoch 4/80
157/157 [==============================] - 130s 826ms/step - loss: 5.9939 - accuracy: 0.3936
Epoch 5/80
157/157 [==============================] - 130s 830ms/step - loss: 4.0910 - accuracy: 0.4088
Epoch 6/80
157/157 [==============================] - 131s 833ms/step - loss: 2.7166 - accuracy: 0.4160
Epoch 7/80
157/157 [==============================] - 131s 835ms/step - loss: 1.5804 - accuracy: 0.4260
Epoch 8/80
157/157 [==============================] - 131s 830ms/step - loss: 1.0354 - accuracy: 0.4376
Epoch 9/80
157/157 [==============================] - 130s 827ms/step - loss: 0.8118 - accuracy: 0.4448
Epoch 10/80
157/157 [==============================] - 131s 832

In [20]:
result = happy_model.evaluate(X_test[:592], Y_test)

19/19 [==============================] - 6s 333ms/step - loss: 0.6289 - accuracy: 0.4966


In [70]:
df1 = pd.read_csv('test.csv')

In [38]:
pred = happy_model.predict(X_test[:592])

In [39]:
result = np.argmax(pred, axis = 1)

In [71]:
df1['class'] = result

In [73]:
df1 ['class'] = df1['class'].replace({ 0:  'Trans-thalamic',1: 'Trans-cerebellum',2 :'Trans-ventricular' ,3 :'Other'})

In [75]:
df1.to_csv('sub.csv', index=False)

In [74]:
df1

,image_name,class
0,Patient01563_Plane3_1_of_3,Trans-thalamic
1,Patient01563_Plane3_2_of_3,Trans-thalamic
2,Patient01563_Plane3_3_of_3,Trans-thalamic
3,Patient01564_Plane3_1_of_2,Trans-cerebellum
4,Patient01564_Plane3_2_of_2,Trans-thalamic
...,...,...
587,Patient01790_Plane3_2_of_2,Trans-thalamic
588,Patient01791_Plane3_1_of_3,Trans-thalamic
589,Patient01791_Plane3_2_of_3,Trans-thalamic
590,Patient01791_Plane3_3_of_3,Trans-thalamic
